## Final Project Submission
* Student name: Charles Owiti, Noel Seda, Michelle Chekwoti , Alex Mwera
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Mildred Jepkosgei

## Business Problem: 

Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

Objectives:
- Analyse movie ratings, number of subscriptions in order to determine the best movies.
- Business recommendations for the new studio for popular genres that will attract large audiences
- Analyse Feedback of users.
- Analysis of high rates genres that could be successful.

The Aim: is to analyse the movie industry to find the gaps in original video content that will allow our new studio to create succesful films.

## Data Understanding

In [13]:
# Importing the relevant libraries
import pandas as pd
import sqlite3

In [14]:
# Connecting to the IMDB database
conn = sqlite3.connect('im.db')
cur = conn.cursor()

In [15]:
# View database tables
tables_query = """SELECT name 
                      AS 'Table Names' 
                      FROM sqlite_master 
                      WHERE type='table';"""

pd.read_sql(tables_query, conn)

Table Names
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers

They are 7 tables in the database and we will focus on the Movie_basics and Movie_ratings.

In [16]:
movie_basics_query = """SELECT * FROM movie_basics ;"""

pd.read_sql(movie_basics_query, conn)

movie_id                                primary_title  \
0       tt0063540                                    Sunghursh   
1       tt0066787              One Day Before the Rainy Season   
2       tt0069049                   The Other Side of the Wind   
3       tt0069204                              Sabse Bada Sukh   
4       tt0100275                     The Wandering Soap Opera   
...           ...                                          ...   
146139  tt9916538                          Kuambil Lagi Hatiku   
146140  tt9916622  Rodolpho Teóphilo - O Legado de um Pioneiro   
146141  tt9916706                              Dankyavar Danka   
146142  tt9916730                                       6 Gunn   
146143  tt9916754               Chico Albuquerque - Revelações   

                                     original_title  start_year  \
0                                         Sunghursh        2013   
1                                   Ashad Ka Ek Din        2019   
2                        The Other Side of the Wind        2018   
3                                   Sabse Bada Sukh        2018   
4                             La Telenovela Errante        2017   
...                                             ...         ...   
146139                          Kuambil Lagi Hatiku        2019   
146140  Rodolpho Teóphilo - O Legado de um Pioneiro        2015   
146141                              Dankyavar Danka        2013   
146142                                       6 Gunn        2017   
146143               Chico Albuquerque - Revelações        2013   

        runtime_minutes                genres  
0                 175.0    Action,Crime,Drama  
1                 114.0       Biography,Drama  
2                 122.0                 Drama  
3                   NaN          Comedy,Drama  
4                  80.0  Comedy,Drama,Fantasy  
...                 ...                   ...  
146139            123.0                 Drama  
146140              NaN           Documentary  
146141              NaN                Comedy  
146142            116.0                  None  
146143              NaN           Documentary  

[146144 rows x 6 columns]

In [17]:
movie_ratings_query = """SELECT * FROM movie_ratings ;"""

pd.read_sql(movie_ratings_query, conn)

movie_id  averagerating  numvotes
0      tt10356526            8.3        31
1      tt10384606            8.9       559
2       tt1042974            6.4        20
3       tt1043726            4.2     50352
4       tt1060240            6.5        21
...           ...            ...       ...
73851   tt9805820            8.1        25
73852   tt9844256            7.5        24
73853   tt9851050            4.7        14
73854   tt9886934            7.0         5
73855   tt9894098            6.3       128

[73856 rows x 3 columns]

The Movie basics table consists of:
- `movie_Id`: The unique id for each movie and the Primary Key.
- `primary_title`: The title of the movie.
- `genres`: The genre(s) of the movie.
  
The Movie ratings table consists of:
- `average_rating`: The average rating given by users.
- `num_votes`: The number of votes a movie has received.
    
Our goal is to analyze the relationship between ratings, votes, and genres.

## Data Preparation and Cleaning

Joining movie_basics table and movie_ratings table which eliminates all movie_basics without ratings on movie_ratings table

In [20]:
data_preparation_query = """SELECT * FROM movie_basics AS mb INNER JOIN movie_ratings as mr ON mr.movie_id = mb.movie_id
;"""

movies_df = pd.read_sql(data_preparation_query, conn)
movies_df

movie_id                    primary_title              original_title  \
0      tt0063540                        Sunghursh                   Sunghursh   
1      tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2      tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3      tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4      tt0100275         The Wandering Soap Opera       La Telenovela Errante   
...          ...                              ...                         ...   
73851  tt9913084                 Diabolik sono io            Diabolik sono io   
73852  tt9914286                Sokagin Çocuklari           Sokagin Çocuklari   
73853  tt9914642                        Albatross                   Albatross   
73854  tt9914942       La vida sense la Sara Amat  La vida sense la Sara Amat   
73855  tt9916160                       Drømmeland                  Drømmeland   

       start_year  runtime_minutes                genres   movie_id  \
0            2013            175.0    Action,Crime,Drama  tt0063540   
1            2019            114.0       Biography,Drama  tt0066787   
2            2018            122.0                 Drama  tt0069049   
3            2018              NaN          Comedy,Drama  tt0069204   
4            2017             80.0  Comedy,Drama,Fantasy  tt0100275   
...           ...              ...                   ...        ...   
73851        2019             75.0           Documentary  tt9913084   
73852        2019             98.0          Drama,Family  tt9914286   
73853        2017              NaN           Documentary  tt9914642   
73854        2019              NaN                  None  tt9914942   
73855        2019             72.0           Documentary  tt9916160   

       averagerating  numvotes  
0                7.0        77  
1                7.2        43  
2                6.9      4517  
3                6.1        13  
4                6.5       119  
...              ...       ...  
73851            6.2         6  
73852            8.7       136  
73853            8.5         8  
73854            6.6         5  
73855            6.5        11  

[73856 rows x 9 columns]

In [22]:
# check the datatypes
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           73052 non-null  object 
 6   movie_id         73856 non-null  object 
 7   averagerating    73856 non-null  float64
 8   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 5.1+ MB


In [ ]:
# Check Null Values
movies_df.isnull().sum()

movie_id              0
primary_title         0
original_title        0
start_year            0
runtime_minutes    7620
genres              804
movie_id              0
averagerating         0
numvotes              0
dtype: int64

Fill null genres with `unknown`

In [38]:
movies_df['genres'] = movies_df['genres'].fillna('unknown')
movies_df

movie_id                    primary_title              original_title  \
0      tt0063540                        Sunghursh                   Sunghursh   
1      tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2      tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3      tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4      tt0100275         The Wandering Soap Opera       La Telenovela Errante   
...          ...                              ...                         ...   
73851  tt9913084                 Diabolik sono io            Diabolik sono io   
73852  tt9914286                Sokagin Çocuklari           Sokagin Çocuklari   
73853  tt9914642                        Albatross                   Albatross   
73854  tt9914942       La vida sense la Sara Amat  La vida sense la Sara Amat   
73855  tt9916160                       Drømmeland                  Drømmeland   

       start_year  runtime_minutes                genres   movie_id  \
0            2013            175.0    Action,Crime,Drama  tt0063540   
1            2019            114.0       Biography,Drama  tt0066787   
2            2018            122.0                 Drama  tt0069049   
3            2018              NaN          Comedy,Drama  tt0069204   
4            2017             80.0  Comedy,Drama,Fantasy  tt0100275   
...           ...              ...                   ...        ...   
73851        2019             75.0           Documentary  tt9913084   
73852        2019             98.0          Drama,Family  tt9914286   
73853        2017              NaN           Documentary  tt9914642   
73854        2019              NaN               unknown  tt9914942   
73855        2019             72.0           Documentary  tt9916160   

       averagerating  numvotes  
0                7.0        77  
1                7.2        43  
2                6.9      4517  
3                6.1        13  
4                6.5       119  
...              ...       ...  
73851            6.2         6  
73852            8.7       136  
73853            8.5         8  
73854            6.6         5  
73855            6.5        11  

[73856 rows x 9 columns]

In [32]:
# Check for Duplicates
movies_df.duplicated().value_counts()

False    73856
dtype: int64

There are no duplicates in the data

## Data Visualization